In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import warnings

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

warnings.simplefilter("ignore")

In [2]:
# Loading the part of the dataset where the sheet information is located
information = pd.read_excel('gov_10a_exp__custom_4037524_spreadsheet.xlsx',
                            sheet_name = None, skiprows = 3, nrows = 4,
                            usecols = 'A, B, C')

# Loading the data
data = pd.read_excel('gov_10a_exp__custom_4037524_spreadsheet.xlsx',
                     sheet_name = None, skiprows = 9)

# Extracting the sheet names from the dataset
keys = list(information.keys())
keys.remove('Summary')
keys.remove('Structure')

information_sheets = {}
dataset = pd.DataFrame()

for key in keys:
    
    df = data[key]
    info_sheet = information[key].T

    # Bringing the sheet information into an appropriate format
    # and storing them into a dictionary
    info_sheet.columns = info_sheet.iloc[0]
    info_sheet.drop(info_sheet.head(2).index,inplace = True)
    info_sheet.reset_index(drop = True, inplace = True)

    information_sheets[key] = info_sheet
    
    # Extracting the category of the information fields
    category_column = info_sheet.columns.values[2]
    category = info_sheet[category_column].iloc[0]

    # Dataset contains some flags, visualized as collumns, which get removed
    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    # Removing the last 5 rows (not needed)
    df.drop(df.tail(5).index,inplace = True)

    # Removing first row since it contains no data
    df.drop(index = 0, inplace = True)

    # Replacing broken header names with their actual name
    df.rename(columns={'TIME': 'Country', 'TIME.1': 'UNIT'}, inplace = True)
    
    
    # Extracting the Year columns
    year_list=list(df.columns)
    year_list.remove('Country')
    year_list.remove('UNIT')

    # Splitting the data based on the measurement unit and also adding the category name
    gdp = df.loc[df['UNIT'] == 'Percentage of gross domestic product (GDP)']
    gdp = pd.melt(gdp, id_vars = ['Country'], value_vars = year_list, var_name = 'Year', value_name = '% GDP', ignore_index=False)
    gdp = gdp.sort_values(by = ['Country', 'Year']).reset_index(drop = True)
    gdp['Category'] = category

    millions = df.loc[df['UNIT'] == 'Million euro']
    millions = pd.melt(millions, id_vars = ['Country'], value_vars = year_list, var_name = 'Year', value_name = 'Million euro', ignore_index=False)
    millions = millions.sort_values(by = ['Country', 'Year']).reset_index(drop = True)

    # Merging the necessary data
    df = millions
    df['% GDP'] = gdp['% GDP']
    df['Category'] = gdp['Category']
    
    dataset = pd.concat([dataset, df])

dataset.reset_index(drop = True, inplace = True)
dataset = dataset.replace(':', np.nan)

dataset

,Country,Year,Million euro,% GDP,Category
0,Austria,2012,163191.9,51.2,Total
1,Austria,2013,167292.1,51.6,Total
2,Austria,2014,174671.6,52.4,Total
3,Austria,2015,176030.0,51.1,Total
4,Austria,2016,179059.0,50.1,Total
...,...,...,...,...,...
25595,Switzerland,2017,55.1,0.0,Social protection n.e.c.
25596,Switzerland,2018,53.7,0.0,Social protection n.e.c.
25597,Switzerland,2019,56.1,0.0,Social protection n.e.c.
25598,Switzerland,2020,59.3,0.0,Social protection n.e.c.


In [5]:
dataset.to_excel('dataset.xlsx')

In [3]:
result = dataset.loc[dataset['Country'] == 'Greece']
result.head(30)

,Country,Year,Million euro,% GDP,Category
130,Greece,2012,106844.0,56.7,Total
131,Greece,2013,112926.0,62.8,Total
132,Greece,2014,89913.0,50.7,Total
133,Greece,2015,95336.0,54.1,Total
134,Greece,2016,87154.0,49.9,Total
135,Greece,2017,85871.0,48.5,Total
136,Greece,2018,87137.0,48.5,Total
137,Greece,2019,87758.0,47.9,Total
138,Greece,2020,98871.0,59.8,Total
139,Greece,2021,NaN,NaN,Total
